In [11]:
import os
import numpy as np
import cv2 as cv
from utils import Utils

In [12]:

menPath = "../Dataset_0-5/men/"
womenPath = "../Dataset_0-5/Women/"

In [13]:

def getThresholdedHand(frame):
    # Convert image to HSV
    hsvim = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # Lower boundary of skin color in HSV
    lower = np.array([0, 48, 80], dtype="uint8")
    # Upper boundary of skin color in HSV
    upper = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv.inRange(hsvim, lower, upper)

    # Gaussian filter (blur) to remove noise
    skinMask = cv.GaussianBlur(skinMask, (17, 17), 0)

    # get thresholded image
    # ret, thresh1 = cv.threshold(
    # skinMask, 100, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    thresh1 = cv.adaptiveThreshold(
        skinMask, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 355, 5)

    return thresh1

In [14]:

# # while True:
# class_dir = os.path.join(menPath, f"{0}")
# imgPath = os.path.join(class_dir, f'{0}_men ({1}).JPG')
# img = cv.imread(imgPath)
# img = getThresholdedHand(img)
# cv.imshow("img", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [15]:
def getMaskedHand2(frame):
        # Convert image to Lab
        rgpImg = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        labImg = cv.cvtColor(rgpImg, cv.COLOR_RGB2LAB)
        # The values of L,A and B channels are  mapped  in  the  range  from  0  up  to  255 
        
        # print(labImg)
        # print(labImg[:5, :5, 1])
        # print(labImg[:5, :5, 2])
        meanL = np.mean(labImg[:, :, 0])
        meanA = np.mean(labImg[:, :, 1])
        meanB = np.mean(labImg[:, :, 2])
        stdL = np.std(labImg[:, :, 0])

        print(meanL, meanA, meanB, stdL)
        

        if meanA + meanB <=256:
                print("first")
                mask =labImg[:, :, 0] <= ( meanL - stdL/3)
                labImg[mask] = np.ones(labImg[mask].shape)*255
        else:
                print("second")
                mask1 = labImg[:, :, 0] <  meanL
                mask2 = labImg[:,:, 2] <  meanB
                mask = mask1 & mask2
                labImg[mask] = np.ones(labImg[mask].shape)*255
        
        rgpImg = cv.cvtColor(labImg, cv.COLOR_LAB2RGB)
        # rgpImg[rgpImg == 255] = 191
        # rgpImg[rgpImg[:,:,1]==70] = 188
        # rgpImg[rgpImg[:,:,0]==0] = 189

        kernel = np.ones((7, 7), np.uint8)
        img_dilation = cv.dilate(rgpImg, kernel, iterations=1)
        img_erosion = cv.erode(img_dilation, kernel, iterations=1)

        # # Gaussian filter (blur) to remove noise
        skinMask = cv.GaussianBlur(img_erosion, (17, 17), 0)
        # greyImg = cv.cvtColor(skinMask,cv.COLOR_RGB2GRAY)
      
        # masked = cv.bitwise_and(greyImg, greyImg, mask=skinMask)
        # maskedBlurred = cv.GaussianBlur(masked, (41, 41), 0)
        return skinMask

In [16]:
for i in range(1, 6):
    out = getMaskedHand2(cv.imread(f"D:/CMP/third_Year/second_semester/pattern Recognition/project/Dataset_0-5/men/0/0_men ({i}).JPG"))
    cv.imshow(f"shadow{i}",  cv.resize(out, (1024, 512)))
    getMasked = Utils.getMaskedHand(out)
    cv.imshow(f"maskedHand{i}", cv.resize(getMasked, (1024, 512)))
    cv.waitKey(0)
    cv.destroyAllWindows()

211.1663156098626 130.63219285501864 134.06508122615526 52.726682961266
second
152.29212463964978 132.82492438003686 134.1406176322445 51.68971866537718
second
148.40268367144634 132.12532141833313 132.93554620520405 52.73099512023074
second


KeyboardInterrupt: 